In [ ]:
import pandas as pd
import altair as alt

cars = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/cars.json'
movies = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/movies.json'
sp500 = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/sp500.csv'
stocks = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/stocks.csv'
flights = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/flights-5k.json'

## 6.4. Deslocar &amp; Ampliar

O gráfico de dispersão da classificação dos filmes está bem desorganizado, dificultando a análise dos pontos em regiões mais densas. Usando as técnicas de interação de _deslocar_ e _ampliar_, podemos inspecionar regiões densas mais de perto.

Vamos começar pensando sobre como podemos expressar deslocamento e ampliação usando seleções do Altair. O que define o "viewport (janela de visualização)" de um gráfico? _Domínios de escala do eixo!_

Podemos mudar o domínio de escala para modificar a faixa de valores dos dados que estão sendo visualizados. Para fazer isso interativamente, podemos vincular uma seleção `interval` para domínios de escala com o código `bind='scales'`. O resultado é que em vez de uma ferramenta que permite selecionar e ampliar um intervalo específico, agora podemos arrastar e ampliar toda a área do gráfico!

_No gráfico abaixo, clique e arraste para deslocar a vista ou role para dar ampliar (escala) na vista. O que você pode descobrir sobre a precisão dos valores de classificações fornecidas?_

In [ ]:
alt.Chart(movies).mark_circle().add_selection(
    alt.selection_interval(bind='scales')
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # use extensão mínima para estabilizar o posicionamento do título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
)

_Ampliando, podemos ver que os valores de classificação tem precisão limitada! As classificações do Rotten Tomatoes são inteiras, enquanto as classificações do IMDB estão truncadas por dezenas. Como resultado, há uma sobre-exibição mesmo quando ampliamos, com vários filmes compartilhando os mesmos valores de classificação._

Lendo o código acima, talvez você note o código `alt.Axis(minExtent=30)` no canal de codificação `y`. O parâmetro `minExtent` garante que uma quantidade mínima de espaço é reservada para marcações e rótulos axiais. Por que fazer isso? Quando deslocamos e ampliamos, os rótulos axiais podem mudar e causar uma deslocação do título do eixo. Colocando uma extensão mínima podemos reduzir movimentos distratores na exibição._Tente mudar o valor de `minExtent`, por exemplo colocando-o no zero e então dimunua o zoom para ver o que acontece quando rótulos de eixo maiores entram na visualização._

O Altair também inclui um atalho para adicionar a um gráfico deslocamento e ampliação. Ao invés de diretamente criar uma seleção, você pode chamar `.interactive()` para fazer o Altair gerar automaticamente uma seleção vinculada às escalas do gráfico:

In [ ]:
alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # use extensão mínima para estabilizar o posicionamento do título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
).interactive()

Por padrão, vínculos de escala para seleções incluem tanto o canal de codificação `x` quanto o canal de codificação `y`. E se queremos limitar a interação de deslocar e ampliar apenas para uma dimensão? Podemos chamar `encodings=['x']` para restringir a seleção apenas para o canal `x`:

In [ ]:
alt.Chart(movies).mark_circle().add_selection(
    alt.selection_interval(bind='scales', encodings=['x'])
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # use extensão mínima para estabilizar o posicionamento do título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
)

_Quando ampliamos ao longo de um único eixo, o formato dos dados visualizados pode mudar, potencialmente afetando nossa percepção das relações nos dados.  [Escolhendo uma razão de aspecto ideal](http://vis.stanford.edu/papers/arclength-banking) é uma preocupação importante para o design da visualização!_

## 6.5. Navegação: Panorama + Detalhe

Quando deslocamos e ampliamos, ajustamos diretamente a janela de visualização de um gráfico. A estratégia de navegação relacionada de _Panorama + Detalhe_ usa, porém, uma exibição geral para mostrar _todos_ os dados, enquanto suporta seleções que deslocam e ampliam numa exibição de um foco separado.

Abaixo temos dois gráficos de área mostrando uma década das flutuações dos preços do índice da ação S&amp;P 500. Inicialmente ambos os gráficos mostram a mesma faixa de dados. _Clique e arraste no gráfico resumido embaixo para atualizar o foco da exibição e examinar períodos de tempo específicos._

In [ ]:
brush = alt.selection_interval(encodings=['x']);

base = alt.Chart().mark_area().encode(
    alt.X('date:T', title=None),
    alt.Y('price:Q')
).properties(
    width=700
)
  
alt.vconcat(
    base.encode(alt.X('date:T', title=None, scale=alt.Scale(domain=brush))),
    base.add_selection(brush).properties(height=60),
    data=sp500
)

Em contraste com o caso anterior deslocar &amp; zoom, aqui não queremos vincular uma seleção diretamente às escalas de um único gráfico interativo. Na verdade, queremos vincular a seleção ao domínio de escala de um _outro_ gráfico. Para fazer isso, atualizamos o canal de codificação `x` para nosso gráfico principal, colocando a propriedade `domain` da escala para referenciar nossa seleção `brush`. Se nenhum intervalo for definido (a seleção está vazia), o Altair ignora a varredura e usa os dados ocultos para determinar o domínio. Quando um intervalo varrido é criado, o Altair usa-o como o `domain` da escala para o gráfico principal.